# Teoría 

El modelo se basa en las reglas de transición que se plantean en el documento de [Ali Khaleghi](../../Documentacion/khaleghi2021.pdf). A partir de esto, se plantea un nuevo modelo para representar las señales de un EEG.


1. Primero se crea una malla de dos dimensiones que va a representar las células. Para representar las conexiones de la malla se usa una matriz de adyacencia, la cual sería de lado \(L \times L\), representando las conexiones de cada neurona.

<details>
<summary>Haz clic aquí para ver el código</summary>

``` 
    std::vector<std::vector<int>> Regular_1(int L, double Inh){
        std::vector<std::vector<int>> Matrix (L*L,std::vector<int>(L*L,0));
        std::random_device rd;
        std::mt19937 gen(1);
        std::uniform_real_distribution<>inh(0.0,1.0);
        double I;
        for (int i = 0; i<L; i++)
        {   
            for (int j = 0; j < L; j++)
            {
                
                int ia=i,ji=j,id=i,jd=j;
                if (ia+1==L)ia=-1;
                if (id==0)id=L;
                if (jd+1==L)jd=-1;
                if (ji==0)ji=L;
                I=inh(gen);if(I<Inh)I=0;else I=1;
                Matrix[i*L+j][i*L+jd+1]=1*(2*I-1);// derecha I=0 -1,I=1 1
                I=inh(gen);if (I<Inh)I=0;else I=1;
                Matrix[i*L+j][i*L+ji-1]=-1*(2*I-1);// izquieda
                I=inh(gen);if (I<Inh)I=0;else I=1;
                Matrix[i*L+j][(ia+1)*L+j]=-1*(2*I-1);// arriba 
                I=inh(gen);if (I<Inh)I=0;else I=1;
                Matrix[i*L+j][(id-1)*L+j]=-1*(2*I-1);// Abajo    

            }
        }
        return Matrix;
    } 

```
</details>

Viendo aquí la creación de un matriz de abjecion de una malla regular, con coneciones con los vecinos, estas conecciones pueden ser tanto exitatorias como inhibidoras, represntadasa como 1 y -1 respectivamente. A continuacion se muestra la representacion de un matriz de abjeccion de una malla de 4 conexion regular

![Representacion de un matriz de abjeccion de una malla de 4 conexion regular](Imagenes/L4.png)


2. Se genera un vector en c++ que represente cada celda(celula), este vector es de tamaño LxL y en este se guarda los estados de las neuronas. Los posibles estados son los siguientes:

$$
S = \Phi_{AP}

\begin{cases}
AP = 0 & \text{Estado de reposo,20\% de probabilidad de estado inicial} \\
AP = 1.2.3.4 & \text{Estado activo, 40\% de probabilidad de estado inicial} \\
AP = 5 & \text{Estado hyperpolarizado, 5\% de probabilidad de estado inicial} \\
AP = 6.7.8.9.10 & \text{Estado refractario, 35\% de probabilidad de estado inicial}
\end{cases}
$$

<details>
<summary>Haz clic aquí para ver el codigo</summary>

```
NeuralNetwork::NeuralNetwork(int L, double P, double inhibidoras, double trest,
                            double trelative, double alpha, int type_matrix)
{
    this->L = L;
    this->P = P;
    this->inhibidoras = inhibidoras;
    this->Trest = trest;
    this->Trelative = trelative;
    this->Alpha = alpha;
    this->L2 = L * L;
    matrix = Matrix(type_matrix);
    AP = std::vector<int>(L2); Vector que guarde los estdos de las celulas. 
    IE = std::vector<bool>(L2);
}
### --------------------------------------------------------

std::mt19937 gen(1); // Mersenne Twister 19937
    std::uniform_real_distribution<> pot(0.0,1.0);
    for (int ix = 0; ix < L2; ix++)
    {
        double potencial =pot(gen);
        if (potencial<0.2){AP[ix]=0;}
        else if (potencial>=0.2 && potencial<0.3){AP[ix]=1;}
        else if (potencial>=0.3 && potencial<0.4){AP[ix]=2;}
        else if (potencial>=0.4 && potencial<0.5){AP[ix]=3;}
        else if (potencial>=0.5 && potencial<0.6){AP[ix]=4;}
        else if (potencial>=0.6 && potencial<0.65){AP[ix]=5;}
        else if (potencial>=0.65 && potencial<0.72){AP[ix]=6;}
        else if (potencial>=0.72 && potencial<0.79){AP[ix]=7;}
        else if (potencial>=0.79 && potencial<0.86){AP[ix]=8;}
        else if (potencial>=0.86 && potencial<0.93){AP[ix]=9;}
        else if (potencial>=0.93){AP[ix]=10;}  
    }
```
</details>

Teniendo en cuenta que cada célula tiene una probabilidad de iniciar en cierto estado, ya con esto se plantea correr la simulación.

3. A cada paso de tiempo, cada célula tiene una regla de transición dependiendo del estado en el que se encuentra.

<details>
<summary>Haz clic aquí para ver el código</summary>


```
double NeuralNetwork::Reglas(int ix){
    Estado Sconexion, Svecino;
    double Ca,Ce,Ci,Ch; 
    Ca=Ce=Ci=Ch=0.0;

    for (int i = 0; i < L2; i++)
    {
        if (matrix[ix][i]==1 || matrix[ix][i]==-1)
        {
            Sconexion=Cual_Estado(i);
            if (Sconexion==Activado )
            {
                if (matrix[ix][i]==-1)Ci++;//se puregunta que tipo de neurona 
                else Ce++;
            }
            else if (Sconexion==hyperpolarizado)
            {
                Ch++;
            }
        }
    }
    Ca=Ce-Ci-Alpha*Ch;// regla de activacion
    return Ca;
}
void NeuralNetwork::Evolucion(){
    Estado St;
    std::vector<int> Aux(L2, 0);
    
    #pragma omp parallel for
    for (int ix = 0; ix < L2; ix++) {
        St = Cual_Estado(ix);
        if (St == Reposo) {
            if (Reglas(ix) >= Trest) Aux[ix] = AP[ix] + 1;
            else Aux[ix] = 0; 
        } else if (St == Activado || St == hyperpolarizado) {
            Aux[ix] = AP[ix] + 1;
        } else if (St == refractario) {
            if (Reglas(ix) >= Trelative) Aux[ix] = 1;
            else {
                if (AP[ix] == 10) Aux[ix] = 0;
                else Aux[ix] = AP[ix] + 1;
            }
        }
    }
    
    AP = Aux;
}
```
</details>

Citando el artículo de Ali Khaleghi:

(a) Activación de una neurona en reposo

Una neurona en reposo se activa cuando la suma de las entradas sinápticas de las neuronas vecinas supera un umbral de reposo \($T_{rest}$\).

$$
P(t,i,j) =
\begin{cases}
AP = AP + 1, & \text{si } Ce - Ci - \alpha Ch \geq T_{rest} \\
AP = AP, & \text{si } Ce - Ci - \alpha Ch < T_{rest} 
\end{cases}
AP = 0.1 \ldots 10
$$

Donde \(P(t,i,j)\) indica el estado de una neurona en la posición \((i,j)\) y en el tiempo \(t\) basado en las fases especificadas de un potencial de acción; \(Ce\), \(Ci\) y \(Ch\) indican el número de neuronas vecinas excitatorias, inhibidoras y en estado de hiperpolarización, respectivamente; y \($\alpha$\) es el coeficiente de hiperpolarización.

(b) Estados de la neurona

Después de la activación, una neurona debe pasar por los estados de disparo, hiperpolarización y periodo refractario, y luego regresar al estado de reposo.

(c) Estado de hiperpolarización

En un estado de hiperpolarización, la neurona no puede generar un potencial de acción subsecuente.

(d) Estado de disparo y periodo refractario

En el estado de disparo, una neurona nunca puede ser activada nuevamente por sus vecinos. Sin embargo, en el periodo refractario, una neurona puede ser activada nuevamente cuando la estimulación de sus vecinos alcanza \($T_{relative}$\) (donde \($T_{relative} > T_{rest}$\)).

$$
P(t,i,j) =
\begin{cases}
AP = AP + 1, & \text{si } Ce - Ci - \alpha Ch \geq T_{relative} \\
AP = AP, & \text{si } Ce - Ci - \alpha Ch < T_{relative}
\end{cases}
$$





